In [1]:
import sys
import os
import warnings
import time

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
warnings.filterwarnings("ignore")

In [2]:
from src.database.db_connection import *
from src.database.preparation import *
from src.evaluation.metrics import *

In [3]:
engine = create_engine_for_db()

In [4]:
query_file_name_expected_Prob = "../../../queries/FP/holoclean/hospital_truth.sql"
query_file_name_actual_Prob = "../../../queries/FP/holoclean/hospital_prob.sql"
algorithm_name = 'PRA'
old_table_name = ''
new_table_name = ''

In [5]:
tids_query = text("""
    SELECT DISTINCT tid
    FROM prototype_fp.hospital_clean
    ORDER BY tid
""")

with engine.connect() as connection:
    tids = connection.execute(tids_query).scalars().all()

In [6]:
total_precision_prob, total_recall_prob,total_coverage_prob, total_noise_prob = 0.0, 0.0, 0.0, 0.0
for tid in tids:
    parameters = {"tid": tid}
    results_Prob,results_with_truth_set_prob = calculate_expected_values(query_file_name_expected_Prob, query_file_name_actual_Prob, algorithm_name,
                                                                             engine,old_table_name,new_table_name,engine2=None,parameters=parameters)
    precision_prob, recall_prob,coverage_prob, noise_prob = summarize_results(results_Prob)
    total_precision_prob += precision_prob
    total_recall_prob += recall_prob
    total_coverage_prob += coverage_prob
    total_noise_prob += noise_prob

avg_precision_prob = total_precision_prob / len(tids)
avg_recall_prob = total_recall_prob / len(tids)
avg_coverage_prob = total_coverage_prob / len(tids)
avg_noise_prob = total_noise_prob / len(tids)

In [7]:
print(f"Average Precision: {avg_precision_prob:.4f}")
print(f"Average Recall: {avg_recall_prob:.4f}")
print(f"Average Coverage: {avg_coverage_prob:.4f}")
print(f"Average Noise: {avg_noise_prob:.4f}")

Average Precision: 1.0000
Average Recall: 0.9990
Average Coverage: 1.0000
Average Noise: 0.0000
